In [3]:
# set up logging
import logging
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s -    %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO,
)

In [4]:
# make deterministic
from utils import set_seed
set_seed(42)

C:\Users\dell\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
import numpy as np
import torch
import torch.nn as nn
from torch.nn import functional as F

In [6]:
import math
from torch.utils.data import Dataset


class CharDataset(Dataset):

    def __init__(self, data, block_size):
        chars = sorted(list(set(data)))
        data_size, vocab_size = len(data), len(chars)
        print(f"data has {data_size:d} characters, {vocab_size:d} unique.")

        self.stoi = { ch: i for i, ch in enumerate(chars) }
        self.itos = { i: ch for i, ch in enumerate(chars) }
        self.block_size = block_size
        self.vocab_size = vocab_size
        self.data = data
    
    def __len__(self):
        return len(self.data) - self.block_size
    
    def __getitem__(self, idx):
        # grab a chunk of (block_size + 1) characters from the data
        chunk = self.data[idx:idx+self.block_size+1]
        # encode every character to an integer
        dix = [self.stoi[s] for s in chunk]
        """
        arrange data and targets so that the first i elements of x
        will be asked to predict the i-th element of y. Notice that
        the eventual language model will actually make block_size
        individual predictions at the same time based on this data,
        so we are being clever and amortizing the cost of the forward
        pass of the network. So for example if block_size is 4, then
        we could e.g. sample a chunk of text "hello", the integers in
        x will correspond to "hell" and in y will be "ello". This will
        then actually "multitask" 4 separate examples at the same time
        in the language model:
        - given just "h", please predict "e" as next
        - given "he" please predict "l" next
        - given "hel" predict "l" next
        - given "hell" predict "o" next
        
        In addition, because the DataLoader will create batches of examples,
        every forward/backward pass during traning will simultaneously train
        a LOT of predictions, amortizing a lot of computation. In particular,
        for a batched input of integers X (B, T) where B is batch size and
        T is block_size and Y (B, T), the network will during training be
        simultaneously training to make B*T predictions, all at once! Of course,
        at test time we can paralellize across batch B, but unlike during training
        we cannot parallelize across the time dimension T - we have to run
        a forward pass of the network to recover the next single character of the 
        sequence along each batch dimension, and repeatedly always feed in a next
        character to get the next one.
        
        So yes there is a big asymmetry between train/test time of autoregressive
        models. During training we can go B*T at a time with every forward pass,
        but during test time we can only go B at a time, T times, with T forward 
        passes.
        """
        x = torch.tensor(dix[:-1], dtype=torch.long)
        y = torch.tensor(dix[1:], dtype=torch.long)
        return x, y

In [7]:
block_size = 128  # spacial extent of the model for its context

In [21]:
# text = open('poem.txt', 'r').read()
text = ('''Sometimes this gnat—that is what he calls himself—knows how to read sometimes he knows how to write; he always knows how to daub. He does
not hesitate to acquire, by no one knows what mysterious mutual
instruction, all the talents which can be of use to the public; from
1815 to 1830, he imitated the cry of the turkey; from 1830 to 1848, he
scrawled pears on the walls. One summer evening, when Louis Philippe
was returning home on foot, he saw a little fellow, no higher than hi
knee, perspiring and climbing up to draw a gigantic pear in charcoal on
one of the pillars of the gate of Neuilly; the King, with that
good-nature which came to him from Henry IV., helped the gamin,
finished the pear, and gave the child a louis, saying: “The pear is on
that also.”19 The gamin loves uproar. A certain state of violence
pleases him. He execrates “the curés.” One day, in the Rue de
l’Université, one of these scamps was putting his thumb to his nose at
the carriage gate of No. 69. “Why are you doing that at the gate?” a
passer-by asked. The boy replied: “There is a curé there.” It was
there, in fact, that the Papal Nuncio lived.In summer, he metamorphoses himself into a frog; and in the evening,
when night is falling, in front of the bridges of Austerlitz and Jena,
from the tops of coal wagons, and the washerwomen’s boats, he hurls
himself headlong into the Seine, and into all possible infractions of
the laws of modesty and of the police. Nevertheless the police keep an
eye on him, and the result is a highly dramatic situation which once
gave rise to a fraternal and memorable cry; that cry which was
celebrated about 1830, is a strategic warning from gamin to gamin; it
scans like a verse from Homer, with a notation as inexpressible as the
eleusiac chant of the Panathenæa, and in it one encounters again the
ancient Evohe. Here it is: “Ohé, Titi, ohééé! Here comes the bobby,
here comes the p’lice, pick up your duds and be off, through the sewer
with you!”


Sometimes this gnat—that is what he calls himself—knows how to read;
sometimes he knows how to write; he always knows how to daub. He does
not hesitate to acquire, by no one knows what mysterious mutual
instruction, all the talents which can be of use to the public; from
1815 to 1830, he imitated the cry of the turkey; from 1830 to 1848, he
scrawled pears on the walls. One summer evening, when Louis Philippe
was returning home on foot, he saw a little fellow, no higher than his
knee, perspiring and climbing up to draw a gigantic pear in charcoal on
one of the pillars of the gate of Neuilly; the King, with that
good-nature which came to him from Henry IV., helped the gamin,
finished the pear, and gave the child a louis, saying: “The pear is on
that also.”19 The gamin loves uproar. A certain state of violence
pleases him. He execrates “the curés.” One day, in the Rue de
l’Université, one of these scamps was putting his thumb to his nose at
the carriage gate of No. 69. “Why are you doing that at the gate?” a
passer-by asked. The boy replied: “There is a curé there.” It was
there, in fact, that the Papal Nuncio lived.

Nevertheless, whatever may be the Voltairianism of the small gamin, if
the occasion to become a chorister presents itself, it is quite
possible that he will accept, and in that case he serves the mass
civilly. There are two things to which he plays Tantalus, and which he
always desires without ever attaining them: to overthrow the
government, and to get his trousers sewed up again.

The gamin in his perfect state possesses all the policemen of Paris,
and can always put the name to the face of any one which he chances to
meet. He can tell them off on the tips of his fingers. He studies their
habits, and he has special notes on each one of them. He reads the
souls of the police like an open book. He will tell you fluently and
without flinching: “Such an one is a _traitor_; such another is very
_malicious_; such another is _great_; such another is _ridiculous_.”
(All these words: traitor, malicious, great, ridiculous, have a
particular meaning in his mouth.) That one imagines that he owns the
Pont-Neuf, and he prevents _people_ from walking on the cornice outside
the parapet; that other has a mania for pulling _person’s_ ears; etc.,
etc.




CHAPTER IX—THE OLD SOUL OF GAUL


There was something of that boy in Poquelin, the son of the
fish-market; Beaumarchais had something of it. Gaminerie is a shade of
the Gallic spirit. Mingled with good sense, it sometimes adds force to
the latter, as alcohol does to wine. Sometimes it is a defect. Homer
repeats himself eternally, granted; one may say that Voltaire plays the
gamin. Camille Desmoulins was a native of the faubourgs. Championnet,
who treated miracles brutally, rose from the pavements of Paris; he
had, when a small lad, inundated the porticos of Saint-Jean de
Beauvais, and of Saint-Étienne du Mont; he had addressed the shrine of
Sainte-Geneviève familiarly to give orders to the phial of Saint
Januarius.

The gamin of Paris is respectful, ironical, and insolent. He has
villainous teeth, because he is badly fed and his stomach suffers, and
handsome eyes because he has wit. If Jehovah himself were present, he
would go hopping up the steps of paradise on one foot. He is strong on
boxing. All beliefs are possible to him. He plays in the gutter, and
straightens himself up with a revolt; his effrontery persists even in
the presence of grape-shot; he was a scapegrace, he is a hero; like the
little Theban, he shakes the skin from the lion; Barra the drummer-boy
was a gamin of Paris; he Shouts: “Forward!” as the horse of Scripture
says “Vah!” and in a moment he has passed from the small brat to the
giant.

This child of the puddle is also the child of the ideal. Measure that
spread of wings which reaches from Molière to Barra.

To sum up the whole, and in one word, the gamin is a being who amuses
himself, because he is unhappy.''')


In [22]:

train_dataset = CharDataset(text, block_size = 128) # one line of poem is roughly 50 characters

data has 5919 characters, 76 unique.


In [23]:
from model import GPT, GPTConfig


mconf = GPTConfig(
    train_dataset.vocab_size,
    train_dataset.block_size,
    n_layer=4,
    n_head=8,
    n_embd=512,
)
model = GPT(mconf)

12/01/2023 22:54:40 - INFO - model -    number of parameters: 12753920


In [29]:
from trainer import Trainer, TrainerConfig

# initialize a trainer instance and kick off training
tconf = TrainerConfig(
    max_epochs=2,
    batch_size=512,
    learning_rate=6e-4,
    lr_decay=True,
    warmup_tokens=512*20,
    final_tokens=2*len(train_dataset)*block_size,
    num_workers=4,
)
trainer = Trainer(model, train_dataset, None, tconf)
trainer.train()

In [27]:
# alright, let's sample some character-level Shaespeare
from utils import sample

context = "Jean Valjean made his appearance"
x = torch.tensor([train_dataset.stoi[s] for s in context], dtype=torch.long)[None, ...].to(trainer.device)
y = sample(model, x, 2000, temperature=1.0, sample=True, top_k=10)[0]
completion = ''.join([train_dataset.itos[int(i)] for i in y])
print(completion)

In [ ]:
Jean Valjean made his appearance in advance of him, which was all
sorts of secrets for soldiers, and of which he would have been greatly
preserved the appearance of superstition, as in contemplation before he
had controlled himself since he had counted on a bickcaxe a spectre
of grape-shot had scaled his face; but he had no longer any need of saying
either “father” to him. A flash of joy illuminated his whole person
who was charged to release him. All these men that they had set out to
follow, nevertheless, only he had meant to take her revenge and had
called her thoughts.—“This are still bleeding.” She shouted to him:—

“Courfeyrac! Courfeyrac! Hohée! He was out! The old man who delivers himself
bonnet.”


The sewer in a severe space, seemed to be appointed the Bishop as
real as he was forced to sell, stepped out, but shutting at his eyes on
the stone, and then he perceived a light.

While the Bishop was behind the trees with the phantoms, as though
enthusiastic over this laughter of a man who is speaking to you. Here is
_boffete_, the bread of the law_, the skin of heaven painful; the
insurgents with their melancholy outline. Their first are they not
conceal themselves. They do not know all these convents, they have
a great joy that I am foolish. I have often been here, and they must not
attempt to disengage the absolute man who desires to force himself to
the situation, had rendered successful, since he had comprehended his
conscience, that all this matter had remained there a people. To see
them, a little tilbury was more of attention than she was marching in the
cafés. And then, over and paid for it with a detachment from the
scene, he cried: “Sister, never a precaution on the part of the culvert
of the Pont de Jéna, where the merchant’s faint still grimacer
has since the preceding evening. He who does not weep does not commune to
enclosure. He will not be mentioned. There is no one who listened to
Courfeyrac:—

“Hey? What’s their mother?”


The prisoner laid down the pen and addressed the fe